<a href="https://colab.research.google.com/github/Zontafor/quantum-software/blob/main/teleporting_teapots_final_project_debugging.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [60]:
!pip install qiskit
!pip install qiskit_nature
!pip install matplotlib
!pip install qiskit-aer
!pip install pylatexenc
!pip install pyscf
!pip install pyscf-forge
!pip install pyscf[all]

from qiskit import QuantumCircuit, QuantumRegister, ClassicalRegister, transpile, assemble
from qiskit_aer import AerSimulator
from qiskit import transpile, assemble
from qiskit.visualization import plot_histogram
from qiskit_nature.second_q.circuit.library import HartreeFock
from qiskit_nature.second_q.mappers import BravyiKitaevMapper
from qiskit_nature.second_q.operators import FermionicOp
from qiskit_nature.second_q.problems import ElectronicStructureProblem
from qiskit_nature.second_q.hamiltonians import ElectronicEnergy
from qiskit_nature.second_q.transformers import FreezeCoreTransformer, BasisTransformer
# from qiskit_nature.second_q.bases import ElectronicBasis
import matplotlib.pyplot as plt
import matplotlib
from pyscf import gto, scf, ao2mo

# Set matplotlib backend
matplotlib.use("Agg")

In [78]:
# Define bond length
bond_length = 0.735

# Create the H2 molecule using PySCF
molecule = gto.M(
    atom=f'H 0 0 0; H 0 0 {bond_length}',
    basis='sto3g',
    unit='Angstrom'
)

# Perform Hartree-Fock calculation
mf = scf.RHF(mol)
mf.kernel()

# Print orbital energies
print("Orbital energies: ", mf.mo_energy)

# Get the integrals in the molecular orbital basis
h1_mo = mf.mo_coeff.T @ mf.get_hcore() @ mf.mo_coeff
eri_mo = ao2mo.restore(1, ao2mo.kernel(mol, mf.mo_coeff), mol.nao_nr())

# Extract number of particles and spatial orbitals
num_particles = (mol.nelectron // 2, mol.nelectron // 2)
num_spatial_orbitals = mf.mo_coeff.shape[1]

# Print properties
print("Number of particles (alpha spin, beta spin): ", num_particles)
print("Number of spatial orbitals: ", num_spatial_orbitals)

# Identify and remove anti-bonding orbitals
# For H2, we have only bonding and anti-bonding orbitals, so we remove the higher energy orbital
occupied_orbitals = slice(0, num_particles[0])
h1_mo_reduced = h1_mo[occupied_orbitals, :][:, occupied_orbitals]
eri_mo_reduced = eri_mo[occupied_orbitals, :, :, :][:, occupied_orbitals, :, :][:, :, occupied_orbitals, :][:, :, :, occupied_orbitals]

# Create the ElectronicEnergy object with reduced integrals
electronic_energy_reduced = ElectronicEnergy.from_raw_integrals(h1_mo_reduced, eri_mo_reduced)

# Create the electronic structure problem
es_problem_reduced = ElectronicStructureProblem(electronic_energy_reduced)

# Check the basis
print(f"Basis of the problem: {es_problem_reduced.basis}")

converged SCF energy = -1.116998996754
Orbital energies:  [-0.58062892  0.67633625]
Number of particles (alpha spin, beta spin):  (1, 1)
Number of spatial orbitals:  2
Basis of the problem: None


In [83]:
# Map the Hamiltonian using Bravyi-Kitaev Mapper
mapper = BravyiKitaevMapper()
qubit_hamiltonian = mapper.map(es_problem.second_q_ops()[0])

# Create Hartree-Fock prepared state
prepared_state = HartreeFock(num_spatial_orbitals, num_particles, mapper)

# Create Quantum Circuit
q = QuantumRegister(prepared_state.num_qubits, 'q')
c = ClassicalRegister(prepared_state.num_qubits, 'c')
circuit = QuantumCircuit(q, c)

# Add the Hartree-Fock circuit
circuit.compose(prepared_state, inplace=True)

# Add measurements
circuit.measure(q, c)

# Draw and show the circuit
#circuit.draw(output='mpl')
# plt.show()
# plt.savefig('hartree_fock_circuit.jpg')

# Simulate the circuit
sim = AerSimulator()
t_circuit = transpile(circuit, sim)
qobj = assemble(t_circuit)
result = sim.run(qobj).result()
counts = result.get_counts(t_circuit)

# Print the prepared state
print(prepared_state)

# Print the molecule and properties
print("Number of particles (alpha spin, beta spin): ", num_particles)
print("Number of spatial orbitals: ", num_spatial_orbitals)

# Print the result of the simulation
print("Simulation result (counts):", counts)

     ┌───┐
q_0: ┤ X ├
     ├───┤
q_1: ┤ X ├
     ├───┤
q_2: ┤ X ├
     └───┘
q_3: ─────
          
Number of particles (alpha spin, beta spin):  (1, 1)
Number of spatial orbitals:  2
Simulation result (counts): {'0111': 1024}


<ipython-input-83-b2d30a4dd46e>:28: DeprecationWarning: Using a qobj for run() is deprecated as of qiskit-aer 0.14 and will be removed no sooner than 3 months from that release date. Transpiled circuits should now be passed directly using `backend.run(circuits, **run_options).
  result = sim.run(qobj).result()
